In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

is_local = False

pdf_to_image_function = "pdftoimage"
image_to_markdown_function = "imagetomarkdown"

if is_local:
    url = "http://localhost:7071/api/"
    
else:
    url = "https://genai-extact-from-pdf.azurewebsites.net/api/"
    
    
pdf_to_image_url = url + pdf_to_image_function
image_to_markdown_url = url + image_to_markdown_function




In [ ]:
import os

# Test 1        
pdf_url1 = os.getenv('PDF_URL_1')
pdf_sas1 = os.getenv('PDF_SAS_1')

pdf_url2 = os.getenv('PDF_URL_2')
pdf_sas2 = os.getenv('PDF_SAS_2')

In [ ]:
import requests
import json



data = {
    "values": [
        {
            "recordId": "record1",
            "data": { 
                "docUrl": pdf_url1,
                "docSAS": pdf_sas1
            }
        },  
        {
            "recordId": "record2",
            "data": { 
                "docUrl": pdf_url1,
                "docSAS": pdf_sas1
            }
        },
    ]
}

response = requests.post(pdf_to_image_url, json=data)
print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.JSONDecodeError:
    print(response.text)

In [ ]:
data={
    "values": [
        {
            "recordId": "record1",
            "data": {
                "image_sas_urls": "XXX"
            }
        },
        {
            "recordId": "record2",
            "data": {
                "image_sas_urls": "XXX"
            }
        }
    ]
}


In [ ]:

response = requests.post(image_to_markdown_url, json=data)
print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.JSONDecodeError:
    print(response.text)
